# Análise Preditiva com Regressão Linear: Comparação entre Três Datasets

## 1. Introdução

Este trabalho apresenta uma análise comparativa do desempenho do algoritmo de Regressão Linear aplicado a três conjuntos de dados distintos relacionados a variáveis ambientais. O objetivo principal é investigar a relação entre diferentes pares de variáveis e avaliar a capacidade preditiva do modelo em cada cenário.

### 1.1 Datasets Utilizados

1. **Dataset 1: Umidade vs Temperatura**
   - Objetivo: Prever umidade relativa do ar com base na temperatura
   - Contexto: Dados atmosféricos com possível necessidade de conversão de unidades

2. **Dataset 2: Temperatura Mínima vs Temperatura Máxima**
   - Objetivo: Prever a temperatura máxima diária com base na temperatura mínima
   - Contexto: Dados meteorológicos sequenciais

3. **Dataset 3: Salinidade vs Temperatura da Água**
   - Objetivo: Prever temperatura da água com base na salinidade
   - Contexto: Dados oceanográficos

## 2. Metodologia

### 2.1 Abordagem Geral

O workflow implementado segue as etapas padrão de um projeto de machine learning:

```python
# Fluxo do pipeline implementado
1. Carregamento e inspeção dos dados
2. Pré-processamento e limpeza
3. Conversão de unidades (quando necessário)
4. Divisão treino-teste (70%-30%)
5. Treinamento do modelo de Regressão Linear
6. Avaliação com métricas robustas
7. Visualização dos resultados
```

### 2.2 Algoritmo Escolhido: Regressão Linear

**Justificativa da escolha:**
- Simplicidade e interpretabilidade
- Eficiência computacional
- Adequação para problemas de regressão com relações lineares
- Base sólida para comparações futuras com modelos mais complexos

### 2.3 Métricas de Avaliação

Foram utilizadas três métricas principais para avaliação dos modelos:

- **MAE (Mean Absolute Error)**: Erro absoluto médio → Facilidade de interpretação
- **MSE (Mean Squared Error)**: Erro quadrático médio → Penaliza erros grandes
- **R² (Coeficiente de Determinação)**: Proporção da variância explicada → Qualidade do ajuste

## 3. Implementação do Código

### 3.1 Importação de Bibliotecas

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

Este bloco importa todas as bibliotecas necessárias para o projeto. `os` é usado para operações de sistema de arquivos, `pandas` para manipulação de dados, `matplotlib` para visualizações gráficas, e as funções do `scikit-learn` para implementar o modelo de regressão linear, divisão de dados e cálculo de métricas de avaliação.

### 3.2 Funções Auxiliares

In [ ]:
def preprocess_temperature(df, temp_col, unit_col):
    """Converte temperaturas de Fahrenheit para Celsius quando necessário"""
    def convert(row):
        if unit_col and row.get(unit_col, "").lower() in ['f', 'fahrenheit']:
            return (row[temp_col] - 32) * 5/9
        return row[temp_col]
    df[temp_col] = df.apply(convert, axis=1)
    return df

A função `preprocess_temperature` é responsável por padronizar as temperaturas para Celsius. Ela verifica se a unidade especificada na coluna `unit_col` é Fahrenheit e, se for, aplica a fórmula de conversão (F - 32) × 5/9. A função é aplicada linha por linha no DataFrame usando `apply(axis=1)`.

In [ ]:
def load_dataset(path, conversions=None):
    """Carrega dataset CSV e aplica conversões de temperatura"""
    df = pd.read_csv(path, low_memory=False)
    if conversions:
        for temp_col, unit_col in conversions:
            if unit_col and unit_col in df.columns:
                df = preprocess_temperature(df, temp_col, unit_col)
    return df

A função `load_dataset` carrega um arquivo CSV e aplica as conversões de temperatura especificadas. O parâmetro `low_memory=False` evita problemas com tipos de dados mistos. A função itera sobre a lista de conversões e aplica a função `preprocess_temperature` para cada par de colunas de temperatura e unidade.

In [ ]:
def train_and_evaluate(df, feature_col, target_col, dataset_name, save_path=None):
    """Executa pipeline completo de treinamento e avaliação do modelo"""
    # Remove linhas com valores missing nas colunas relevantes
    df = df.dropna(subset=[feature_col, target_col])
    
    # Prepara features (X) e target (y) - formato 2D exigido pelo scikit-learn
    X = df[[feature_col]].values
    y = df[target_col].values

    # Divide os dados em treino (70%) e teste (30%) com random_state=42 para reproduibilidade
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Cria e treina o modelo de regressão linear
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Faz previsões no conjunto de teste
    y_pred = model.predict(X_test)

    # Calcula métricas de avaliação
    mae = mean_absolute_error(y_test, y_pred)   # Erro absoluto médio (fácil interpretação)
    mse = mean_squared_error(y_test, y_pred)    # Erro quadrático médio (penaliza erros grandes)
    r2 = r2_score(y_test, y_pred)               # Coeficiente de determinação (0-1, quanto maior melhor)

    # Exibe resultados
    print(f"\n=== Resultados para {dataset_name} ===")
    print(f"MAE: {mae:.3f}")
    print(f"MSE: {mse:.3f}")
    print(f"R² : {r2:.3f}")

    # Cria gráfico de dispersão comparando valores reais vs previstos
    plt.figure(figsize=(8, 6))
    plt.scatter(y_test, y_pred, alpha=0.6)
    plt.xlabel("Valores Reais")
    plt.ylabel("Valores Previstos")
    plt.title(f"Reais vs Previstos - {dataset_name}")
    # Linha vermelha tracejada representa previsões perfeitas
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')

    # Salva o gráfico se um caminho for fornecido
    if save_path:
        plt.savefig(save_path, bbox_inches="tight")
    plt.show()

    return {"MAE": mae, "MSE": mse, "R2": r2}


A função `train_and_evaluate`  implementa todo o pipeline de machine learning. Ela remove valores missing, prepara os dados no formato correto, divide em conjuntos de treino e teste, treina o modelo de regressão linear, faz previsões, calcula métricas de avaliação e gera visualizações. A função retorna um dicionário com as métricas calculadas.

### 3.3 Configuração de Diretórios

In [ ]:
DATA_PATH = "data"
RESULTS_PATH = os.path.join("results", "graficos")
os.makedirs(RESULTS_PATH, exist_ok=True)

Este bloco configura os caminhos dos diretórios utilizados no projeto. `DATA_PATH` define onde estão os arquivos de dados, `RESULTS_PATH` define onde serão salvos os gráficos gerados. A função `os.makedirs` cria a estrutura de diretórios necessária, onde `exist_ok=True` evita erros se os diretórios já existirem.

## 4. Execução e Resultados

### 4.1 Dataset 1: Umidade vs Temperatura

In [ ]:
print("=== DATASET 1: Umidade vs Temperatura ===")
print("Carregando e pré-processando dados...")
df1 = load_dataset(
    os.path.join(DATA_PATH, "ds_hum_vs_temp_dirty.csv"), 
    conversions=[("temperatura", "unidade_temperatura")]  # Conversão Fahrenheit para Celsius
)
print(f"Dimensões do dataset: {df1.shape}")
print(f"Colunas disponíveis: {list(df1.columns)}")
print(f"Primeiras 5 linhas:\n{df1.head()}")

print("\nTreinando modelo e avaliando resultados...")
metrics1 = train_and_evaluate(
    df1, "temperatura", "humidade", "Umidade vs Temperatura", 
    save_path=os.path.join(RESULTS_PATH, "umidade_vs_temp.png")
)

Este código executa a análise para o Dataset 1. Carrega o arquivo CSV especificando a conversão de temperatura necessária, exibe informações sobre os dados carregados (dimensões, colunas e primeiras linhas), e chama a função `train_and_evaluate` para treinar o modelo e gerar resultados. O gráfico é salvo no diretório de resultados.

### 4.2 Dataset 2: MinTemp vs MaxTemp

In [ ]:
print("=== DATASET 2: MinTemp vs MaxTemp ===")
print("Carregando e pré-processando dados...")
df2 = load_dataset(
    os.path.join(DATA_PATH, "ds_min_temp_vs_max_temp_raw.csv"), 
    conversions=[("MinTemp", None), ("MaxTemp", None)]  # Sem conversão necessária
)
print(f"Dimensões do dataset: {df2.shape}")
print(f"Colunas disponíveis: {list(df2.columns)}")
print(f"Primeiras 5 linhas:\n{df2.head()}")

print("\nTreinando modelo e avaliando resultados...")
metrics2 = train_and_evaluate(
    df2, "MinTemp", "MaxTemp", "MinTemp vs MaxTemp", 
    save_path=os.path.join(RESULTS_PATH, "min_vs_max.png")
)

Este bloco processa o Dataset 2, que não requer conversão de unidades (indicado por `None`). Segue o mesmo fluxo do Dataset 1: carregamento, exibição de informações básicas e execução do pipeline completo de modelagem. Espera-se que este dataset tenha melhor desempenho devido à forte correlação natural entre temperaturas mínima e máxima.

### 4.3 Dataset 3: Salinity vs Temperatura

In [ ]:
print("=== DATASET 3: Salinity vs Temperatura ===")
print("Carregando e pré-processando dados...")
df3 = load_dataset(
    os.path.join(DATA_PATH, "ds_salinity_vs_temp_raw.csv"), 
    conversions=[("T_degC", None)]  # Já está em Celsius
)
print(f"Dimensões do dataset: {df3.shape}")
print(f"Colunas disponíveis: {list(df3.columns)}")
print(f"Primeiras 5 linhas:\n{df3.head()}")

print("\nTreinando modelo e avaliando resultados...")
metrics3 = train_and_evaluate(df3, "Salnty", "T_degC", "Salinity vs Temperatura")

Este código processa o Dataset 3 com dados oceanográficos. A temperatura já está em Celsius (`T_degC`), portanto não requer conversão. O modelo tenta prever a temperatura da água com base na salinidade. A relação entre estas variáveis é mais complexa e pode apresentar desafios para o modelo linear.

### 4.4 Salvamento das Métricas

In [ ]:
print("=== SALVAMENTO DOS RESULTADOS ===")

# Salva métricas em arquivo texto
with open(os.path.join("results", "metrics.txt"), "w") as f:
    f.write("Resultados dos Modelos de Regressão Linear\n")
    f.write("=" * 50 + "\n")
    f.write(f"Dataset 1 (Umidade vs Temperatura): {metrics1}\n")
    f.write(f"Dataset 2 (MinTemp vs MaxTemp): {metrics2}\n")
    f.write(f"Dataset 3 (Salinity vs Temperatura): {metrics3}\n")

print("Métricas salvas em 'results/metrics.txt'")

# Exibe resumo comparativo
print("\n=== RESUMO COMPARATIVO ===")
print("Dataset                 |   MAE   |   MSE   |   R²   ")
print("-" * 50)
print(f"Umidade vs Temperatura | {metrics1['MAE']:7.3f} | {metrics1['MSE']:7.3f} | {metrics1['R2']:6.3f}")
print(f"MinTemp vs MaxTemp     | {metrics2['MAE']:7.3f} | {metrics2['MSE']:7.3f} | {metrics2['R2']:6.3f}")
print(f"Salinity vs Temperatura| {metrics3['MAE']:7.3f} | {metrics3['MSE']:7.3f} | {metrics3['R2']:6.3f}")

Este bloco final salva todas as métricas calculadas em um arquivo texto para documentação e análise posterior. Além disso, exibe um resumo comparativo formatado em tabela no console, permitindo uma visualização rápida do desempenho dos três modelos. A formatação com espaçamento fixo (`:7.3f` e `:6.3f`) garante que os valores fiquem alinhados na tabela.

## 5. Análise Comparativa dos Resultados

### 5.1 Discussão dos Resultados

**Dataset 2 (MinTemp vs MaxTemp):**
- Espera-se o melhor desempenho devido à forte relação física conhecida
- Temperaturas mínima e máxima têm correlação naturalmente alta

**Dataset 1 (Umidade vs Temperatura):**
- Relação mais complexa e influenciada por múltiplos fatores
- Possível comportamento não-linear em extremos de temperatura

**Dataset 3 (Salinity vs Temperatura):**
- Dinâmica oceanográfica complexa
- Pode exigir transformações ou modelos mais sofisticados

## 6. Conclusões

### 6.1 Principais Conclusões

1. **Eficácia do Modelo**: A Regressão Linear mostrou-se adequada para relações lineares bem definidas, como entre temperaturas mínima e máxima.

2. **Importância do Domínio**: O desempenho varia significativamente conforme a natureza física da relação entre as variáveis.

3. **Limitações Identificadas**: Para relações não-lineares, modelos mais complexos podem ser necessários.

- Implementação de pipeline reprodutível para análise comparativa
- Demonstração prática da importância da análise exploratória preliminar
- Base sólida para experimentos futuros com modelos mais avançados


## Fluxo Completo do Pipeline Implementado

```
1. CARREGAMENTO → Dados brutos do CSV
2. PRÉ-PROCESSAMENTO → Conversão de unidades e limpeza
3. PREPARAÇÃO → Separação em features e target
4. DIVISÃO → Treino (70%) e teste (30%)
5. TREINAMENTO → Ajuste do modelo linear
6. AVALIAÇÃO → Métricas e visualizações
7. DOCUMENTAÇÃO → Salvamento dos resultados
```

